##### 【 의사결정나무 알고리즘 - DT 】
- iris(붓꽃) 품종 분류 모델
- 학습종류 : 지도학습 - 분류
- 학습방법 : DT기반 분류

[1] 모듈 로딩 및 데이터 준비 <hr>
- [1-1] 모듈 로딩
- [1-2] 데이터 준비

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
DATA_FILE = '../Data/iris.csv'

df = pd.read_csv(DATA_FILE)
df.head(3)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


[2] 학습 준비<hr>
- [2-1] 피쳐/타겟 분리
- [2-2] 학습용/테스트용 분리
- [2-3] 수치형 피쳐 스케일링

In [ ]:
featureDF = df[df.columns[:-1]]
targetSR  = df[df.columns[-1]]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(featureDF, targetSR, test_size=0.2, random_state=42, )